In [1]:
import sys
import random
import pickle
from os.path import join

sys.path.append('..')

import numpy as np
import networkx as nx

from cnrg.VRG import VRG as VRG
from cnrg.LightMultiGraph import convert
from cnrg.LightMultiGraph import LightMultiGraph as LightMultiGraph
from cnrg.generate import generate_graph
from cnrg.globals import find_boundary_edges
from cnrg.generate import generate_graph

import matplotlib.pyplot as plt
import matplotlib_inline
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
sys.path.append('../src')
from data import load_data
from decomposition import decompose, decompose_component
from adjoin_graph import update_grammar

# prelims

In [3]:
loaded = load_data(dataname='email-eucore', lookback=0)
graphs = [g for _, g in loaded]
times = [t for t, _ in loaded]

In [4]:
t = 13
g = graphs[t]
h = graphs[t + 1]

In [11]:
grammar = decompose(g, time=t)
grammar

graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 243(243) mdl: 20_058.7 bits

In [9]:
igrammar = update_grammar(grammar, g, h, time=t+1, signature='i', verbose=True)
display(igrammar, igrammar.ll)

additions: 100%|██████████| 63/63 [00:00<00:00, 13852.02it/s]


graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 245(245) mdl: 30_337.2 bits

-10.719007221086263

In [12]:
jagrammar = update_grammar(grammar, g, h, time=t+1, signature='ja', verbose=True)
display(jagrammar, jagrammar.ll)

additions: 100%|██████████| 11/11 [00:00<00:00, 16869.23it/s]


graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 255(255) mdl: 30_687.8 bits

-10.663475290014881

In [13]:
jbgrammar = update_grammar(grammar, g, h, time=t+1, signature='jb', verbose=True)
display(jbgrammar, jbgrammar.ll)

additions: 100%|██████████| 11/11 [00:00<00:00, 19160.03it/s]


graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 253(253) mdl: 30_588.5 bits

-10.6633817512079

# generation

In [ ]:
base_n = g.order()
target_n = len(set(g.nodes()) | set(h.nodes()))
target_e = len(set(g.edges()) | set(h.edges()))
print(target_n, target_e, '|', g.order(), g.size(), '|', h.order(), h.size())

In [ ]:
base_g, base_ro = generate_graph(base_n, grammar, merge_iso_rules=False)
base_g

In [ ]:
ig, iro = generate_graph(target_n, igrammar, merge_iso_rules=False)
ig

In [ ]:
jag, jaro = generate_graph(target_n, jagrammar, merge_iso_rules=False)
jag

In [ ]:
jbg, jbro = generate_graph(target_n, jbgrammar, merge_iso_rules=False)
jbg

In [ ]:
for rule, pidx, anode in jbgrammar:
    if rule.lhs != (ll := sum(d['b_deg'] for v, d in rule.graph.nodes(data=True))):
        print(rule.lhs, ll, rule, pidx, anode)

In [ ]:
for rule, _, _ in jbgrammar:
    try:
        if rule.branch:
            print(rule)
    except:
        continue

In [ ]:
jbgrammar.root